<a href="https://colab.research.google.com/github/jeibloo/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [16]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-07-22 19:29:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2019-07-22 19:29:15 (620 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [18]:
!head house-votes-84.data

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?


In [0]:
### SETUP
import pandas as pd
import numpy as np

column_names = ['party','hInfant_bill','waterProject_bill','budgetAdopt_bill',
                'physc_payFrz_bill','elSalv_bill','relgGrp_school_bill',
                'antiSat_ban_bill','contrasAid_bill','mxMis_bill','immigrt_bill',
                'synfuels_cut_bill','education_bill','superfundSue_bill',
                'crime_bill','dutyFree_bill','exportAdmin_SA_bill']
# It came with no headers
df = pd.read_csv('./house-votes-84.data',names=column_names)
# Index is good as numerical

In [22]:
print(df.shape)
df.head(3)

(435, 17)


,party,hInfant_bill,waterProject_bill,budgetAdopt_bill,physc_payFrz_bill,elSalv_bill,relgGrp_school_bill,antiSat_ban_bill,contrasAid_bill,mxMis_bill,immigrt_bill,synfuels_cut_bill,education_bill,superfundSue_bill,crime_bill,dutyFree_bill,exportAdmin_SA_bill
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n


In [0]:
df = df.replace('?',np.nan)

In [41]:
# Make copy of dataframe in case I mess up.
df_clean = df.copy()

## Change yes to 1 no to 0
def convert(data_obj,ugly,good):
  data_obj = data_obj.replace(ugly,good) 
  return data_obj

# Apply it twice
df_clean = convert(df_clean,'y',1)
df_clean = convert(df_clean,'n',0)

df_clean.head(5)

,party,hInfant_bill,waterProject_bill,budgetAdopt_bill,physc_payFrz_bill,elSalv_bill,relgGrp_school_bill,antiSat_ban_bill,contrasAid_bill,mxMis_bill,immigrt_bill,synfuels_cut_bill,education_bill,superfundSue_bill,crime_bill,dutyFree_bill,exportAdmin_SA_bill
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


#Data is relatively clean, I believe

This will use a 2-sample t-test because it's across two groups (as per instructions)

Null hypothesis: No one is different, everyone voted in the exact same way like robots.

REMINDER: 

What is **t-stat**?: ~Roughly the # of std. dev. away from mean, that coorresponds to the differences in means we've observed given their sample sizes.

What is **p-val**?: ~Probability that what we have observed (the t-statist that we get) could have happened by random chance.

In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

democrat = df_clean[df_clean.party == 'democrat']
republican = df_clean[df_clean.party == 'republican']

def p_value(groupA,groupB,colX):
  return ttest_ind(groupA[colX],groupB[colX],nan_policy='omit')

### 2). 3).
### Before:

1) Null Hypo (boring/default state)
- the two parties are voting the same $\mu_{1} = \mu_{2}$ 

2) Alt. Hypo (opposite of null hypo)
- the two parties are voting vastly differently

3) Confidence Level: 95%

**Confidence Level of 95%** (reach incorrect conclusion only 5% of the time)

p < 1 - Confidence Level

1- Confidence Level = $\alpha$

I REJECT THE NULL HYPO when my P-value < $\alpha$ (1 - Confidence Level)

In [70]:
## 2. Use hypo testing to find issue Demz support more than Repubs w/ p < 0.01
p_value(democrat,republican,'hInfant_bill')
### p-value shows that it is incredibly super duper unlikely that they voted 
### different by chance. So I reject my null hypothesis. So alternative hypothesis
### is at work here.

Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

### 2).
### After:

4) T-stat: 9.2053

5) P-value: 1.613e-18

### Interpretation:

Due to the calculating t-stat of 9.2053 which corresponds to 1.613e-18, we reject the null hypo that the parties are voting in the same way, and suggest the alternative hypo that the parties are voting super different on this bill.



In [67]:
## 3. Use hypo testing to find issue Repubs support more than Demz w/ p < 0.01
p_value(democrat,republican,'contrasAid_bill')

Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

### 3).
### After:

4) T-stat: 18.0521

5) P-value: 2.825e-54

### Interpretation:

Due to the calculating t-stat of 18.0521 which corresponds to 2.825e-54, we reject the null hypo that the parties are voting in the same way, and suggest the alternative hypo that the parties are voting extremely differently on this bill.



---



### 4).
### Before:

1) Null Hypo (boring/default state)
- the two parties are voting differently $\mu_{1} = \mu_{2}$ 

2) Alt. Hypo (opposite of null hypo)
- the two parties are voting similiarly on bills that have no public interest

3) Confidence Level: 95%

**Confidence Level of 95%** (reach incorrect conclusion only 5% of the time)

p < 1 - Confidence Level

1- Confidence Level = $\alpha$

I REJECT THE NULL HYPO when my P-value < $\alpha$ (1 - Confidence Level)

In [68]:
## 4. Use hypo testing to find Repubs and Demz supporting same bill w/ p > 0.01
p_value(democrat,republican,'waterProject_bill')
### I totally accept my imaginary alt hypothesis that boring bills are bipartisan.

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

### After:

4) T-stat: -0.0890

5) P-value: 0.929

### Interpretation:


Due to the calculating t-stat of -0.0890 which corresponds to 0.929, we reject the null hypo that the parties are voting in a different way, and suggest the alternative hypo that the parties are voting the same way on bills that have no partisan slant.